# Plot CrossedWires

This Kotlin Notebook uses the `CrossedWires` implementation from the main source set and writes the original input's wiring to a ".dot" file, `wiredGates.dot`. That file can be rendered using GraphViz (DOT, https://graphviz.org).

First, read and parse the input for lists of `Gate`s and `Wire`s.

In [1]:
import de.ronny_h.aoc.AdventOfCode
import de.ronny_h.aoc.year24.day24.CrossedWires

val crossedWires = CrossedWires()
val input = crossedWires.readInput()
val gates = crossedWires.parseGates(input)
val wires = crossedWires.parseWires(input)

Add IDs to the Gates and Wires and construct some maps for fast lookup.

In [2]:
data class GateWithId(val id: String, val in1: String, val in2: String, val op: String, val out: String)
data class WireWithId(val id: String, val name: String)

val gatesWithId = gates.mapIndexed { i, g -> GateWithId("$i", g.in1, g.in2, g.operation.toString(), g.out) }
val wiresWithId = wires.mapIndexed { i, w -> WireWithId("${i + gates.size}", w.name) }
val gatesById = gatesWithId.associateBy { it.id }
val wiresById = wiresWithId.associateBy { it.id }
val gatesOutLookup = gatesWithId.associateBy { it.out }
val gatesInLookup = gatesWithId.fold(emptyMap<String, GateWithId>()) { acc, gate ->
    (acc + (gate.in1 to gate)) + (gate.in2 to gate)
}
val wiresLookup = wiresWithId.associateBy { it.name }

Construct a pair of lists so that for each index `i` in both lists there is a wiring from `first[i]` to `second[i]`.

`first` is a list of IDs of input wires or gates, `second` is a list of IDs of gates or output wires.

In [3]:
fun wireTheGates(): Pair<List<String>, List<String>> {
    val from = mutableListOf<String>()
    val to = mutableListOf<String>()

    fun wireInput(gateId: String, inWireName: String) {
        val inGate = gatesOutLookup.get(inWireName)
        if (inGate == null) {
            // in is a wire
            from.add(wiresLookup.getValue(inWireName).id)
        } else {
            // in is another gate
            from.add(inGate.id)
        }
        to.add(gateId)
    }

    fun wireOutput(gateId: String, outWireName: String) {
        if (gatesInLookup.get(outWireName) == null) {
            // output is a wire; other gates are handled via wireInput()
            from.add(gateId)
            to.add(outWireName)
        }
    }

    for (g in gatesWithId) {
        wireInput(g.id, g.in1)
        wireInput(g.id, g.in2)
        wireOutput(g.id, g.out)
        check(from.size == to.size) { "different sizes: from: '$from', to: $to" }
    }
    return from to to
}

val wiredGateIds = wireTheGates()

Write the data to a .dot file that can be interpreted by Graphviz.

In [4]:
import java.io.File

File("wiredGates.dot").printWriter().use { out ->
    out.println("digraph {")
    out.println("  edge [fontname=Arial fontsize=9 arrowhead=vee arrowsize=0.6]")
    out.println("  node [fontname=Arial shape=plain]")
    val gatesCount = gatesWithId.size
    val outWires = gatesWithId.filter { gatesInLookup[it.out] == null }.map { it.out }
    val inXColor = "\"#4A88D4\""
    val inYColor = "\"#784AD4\""
    val outZColor = "\"#48B540\""

    wiresWithId.forEach { wire ->
        val color = when {
            wire.name.startsWith("x") -> inXColor
            wire.name.startsWith("y") -> inYColor
            wire.name.startsWith("z") -> outZColor
            else -> "grey"
        }
        out.println("  ${wire.id} [label=${wire.name} fontcolor=$color color=$color]")
    }
    outWires.forEach {
        out.println("  ${it} [label=${it} fontcolor=$outZColor color=$outZColor]")
    }
    gatesWithId.forEach { gate ->
        out.println("  ${gate.id} [label=\"${gate.op}\"]")
    }

    wiredGateIds.first.forEachIndexed { i, fromId ->
        val toId = wiredGateIds.second[i]
        val fromGate = gatesById[fromId]
        val attributes = if (fromGate != null) {
            if (fromGate.out.startsWith("z")) {
                " [color=$outZColor]"
            } else {
                // the "from" and "to" nodes are gates -> label their connection
                " [label=${fromGate.out}]"
            }
        } else {
            // the "from" node is an input
            if (wiresById.getValue(fromId).name.startsWith("x")) {
                " [color=$inXColor]"
            } else {
                " [color=$inYColor]"
            }
        }
        out.println("  $fromId -> $toId$attributes")
    }

    out.println("}")
}

The graph can be generated by calling

```bash
dot wiredGates.dot -Tsvg -o wiredGates.svg
```

It looks like this:

![A Graph showing the wired gates with inputs and outputs](./wiredGates.svg)